https://www.curiousily.com/posts/create-dataset-for-sentiment-analysis-by-scraping-google-play-app-reviews-using-python/

In [3]:
!pip install google_play_scraper

  Created wheel for google-play-scraper: filename=google_play_scraper-0.0.3.0-cp36-none-any.whl size=21953 sha256=342364d5de29d5314b8cef3eaffd1bfb25f08cf04de7fadb63ef95b78b447af9
  Stored in directory: C:\Users\aosika\AppData\Local\pip\Cache\wheels\34\0d\1a\cd5993428dc7c5710ed5b2201ba7c0f3f14f80c951e4bde14e
Successfully built google-play-scraper


ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\aosika\AppData\Local\Continuum\anaconda3\envs\learn-env\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _dep_map
    return self.__dep_map
  File "C:\Users\aosika\AppData\Local\Continuum\anaconda3\envs\learn-env\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aosika\AppData\Local\Continuum\anaconda3\envs\learn-env\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3003, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\aosika\AppData\Local\Continuum\anaconda3\envs\learn-env\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

Durin

In [4]:
import json
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
from google_play_scraper import Sort, reviews, app
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)

## Why does THIS work?

In [16]:
app_packages = [
  'com.anydo',
  'com.todoist',
  'com.ticktick.task',
  'com.habitrpg.android.habitica',
  'cc.forestapp',
  'com.oristats.habitbull',
  'com.levor.liferpgtasks',
  'com.habitnow',
  'com.microsoft.todos',
  'prox.lab.calclock',
  'com.gmail.jmartindev.timetune',
  'com.artfulagenda.app',
  'com.tasks.android',
  'com.appgenix.bizcal',
  'com.appxy.planner'
]

## ....and NOT this?

In [20]:
# app_packages = [
#     'com.offerup',
#     'com.letgo',
#     'com.carfax',
#     'com.marketPlace',
#     'com.perkwiz',
    
# ]

In [21]:
app_infos = []

for ap in tqdm(app_packages):
    info = app(ap, lang='en', country='us')
    del info['comments']
    app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


In [26]:
def print_json(json_object):
    json_str = json.dumps(
        json_object,
        indent=2,
        sort_keys=True,
        default=str)
    print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [29]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 and up",
  "appId": "com.offerup",
  "containsAds": null,
  "contentRating": "Everyone",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "<b>Buy. Sell. Simple.</b>\r\n\r\nJoin the largest mobile marketplace for local buyers and sellers! \r\n\r\nOfferUp makes it easy to find great deals on the things you want and make money on the things you want to sell. Ditch the Classified Ads and garage sales -- this is the best way to buy and sell in your community or neighborhood with people you can trust.\r\n\r\n<b>How it works:</b>\r\n\r\n\u2022 Buy or sell anything; easily offer up your items for sale in 30 seconds. \r\n\u2022 Find great deals and discounts on clothes, shoes, furniture, cell phones, electronics, baby & kids items, sports equipment, used cars, home goods and more.\r\n\u2022 Use reputation features like ratings and profiles to see who you\u2019re dealing with and build trust

In [32]:
app_reviews = []
for ap in tqdm(app_packages):
    for score in list(range(1, 6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
            ap,
            lang='en',
            country='us',
            sort=sort_order,
            count= 200 if score == 3 else 100,
            filter_score_with=score)
    for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
        app_reviews.extend(rvs)

100%|██████████| 1/1 [00:07<00:00,  7.78s/it]


In [34]:
len(app_reviews)

10000

In [35]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)

In [36]:
app_reviews_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOGoRl6Fb6bMyKVB1VldyR78Ky2Su2OrIKpjfHY...,Jorge Luzuriaga,https://lh3.googleusercontent.com/-VLlPlAcBBhQ...,"People want money for there used or new stuff,...",5,0,3.59.0,2020-06-29 20:37:01,None,None,newest,com.offerup
1,gp:AOqpTOEbdNzNeNQCgJYvMQ0k-NapnIWZmfd5L5S_ct5...,Paul Bustamante,https://lh3.googleusercontent.com/-KhkckqJs4nI...,Very helpful,5,0,3.59.0,2020-06-29 20:18:54,None,None,newest,com.offerup
2,gp:AOqpTOGU1HTMl2AJ9xMupPdfGStj0Mid0q8Qa2BmVXm...,Miguel Trejo,https://lh3.googleusercontent.com/-Qv_9hJBgd7Y...,"Love this app, very helpful and there's someth...",5,0,3.59.0,2020-06-29 19:58:24,None,None,newest,com.offerup
3,gp:AOqpTOFyTN17MMi7CLL5Cnjji_w2qo09ar1opzU_jWO...,Rosemary,https://lh3.googleusercontent.com/a-/AOh14GhcL...,Best app so far. No hidden fees and easy to se...,5,0,3.59.0,2020-06-29 19:57:12,None,None,newest,com.offerup
4,gp:AOqpTOHOgtJa2u5LrbJ2S-a0jWi1M0ytUe3FpkE-41h...,Tamara Eggiman,https://lh3.googleusercontent.com/a-/AOh14Gg__...,Always find great things...,5,0,3.59.0,2020-06-29 19:34:05,None,None,newest,com.offerup
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,gp:AOqpTOEeNoaMuaJD5ThIIjtcHizQ0-0ezpDcHljYpOR...,Christian Saldana,https://lh3.googleusercontent.com/-RY1IELmvh20...,😃,5,0,3.58.0,2020-06-28 19:07:50,None,None,newest,com.offerup
9996,gp:AOqpTOHISuhfU3cVLykvWyVQtQPlTDp5C6KaH0Ct_fA...,brenda lopez,https://lh3.googleusercontent.com/a-/AOh14GhU7...,Easy to navigate,5,0,3.59.0,2020-06-28 18:38:07,None,None,newest,com.offerup
9997,gp:AOqpTOFfpcd2ygVQTGieZErZfo4K_LhjvlOGv6l9CVq...,Monda Jackson,https://lh3.googleusercontent.com/a-/AOh14GiFI...,Very useful and I've purchased many items and ...,5,0,3.58.0,2020-06-28 18:37:03,None,None,newest,com.offerup
9998,gp:AOqpTOGsdZKPVA45DuoNoQKJDTZlfmMhFZWPAm--t9k...,Carlos Diaz,https://lh3.googleusercontent.com/-1VDF94yVFNQ...,Good products and good selling tool,5,0,3.58.0,2020-06-28 18:35:00,None,None,newest,com.offerup


In [39]:
app_reviews_df['score'].value_counts()

5    10000
Name: score, dtype: int64

In [38]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
reviewId                10000 non-null object
userName                10000 non-null object
userImage               10000 non-null object
content                 10000 non-null object
score                   10000 non-null int64
thumbsUpCount           10000 non-null int64
reviewCreatedVersion    9300 non-null object
at                      10000 non-null datetime64[ns]
replyContent            0 non-null object
repliedAt               0 non-null object
sortOrder               10000 non-null object
appId                   10000 non-null object
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 937.6+ KB


## ...then how do dig in to the reviews????